In [2]:
# Alex Shah
# 5/13/2020
# Download x many videos from all subscribed channels

# 0. pytube download youtube videos
# 1. set up docker
# 2. read subscription list and get channel url's
# 3. get recent video urls from channel url
# 4. Download recent videos per channel
# 5. simpleserver or flask to host videos (viewer?)
# 6. write tests
# 7. deploy test

!pip install pytube3
from pytube import YouTube
import requests
import json
import os
from multiprocessing import Pool
import time

In [0]:
path = os.path.abspath(os.getcwd())
pool = Pool(processes=4)
json_data = dict()
subs = dict()
vidInfo = dict()

In [0]:
def getChannelVids(name, url, GRAB_LAST = 0):
    vidURLs = dict()   
    # path for channel
    pathName = path + "/videos/" + name
    # Download channel url, attribute "content" is the list of videos
    page = requests.get(url).content
    data = str(page).split(' ')
    item = 'href="/watch?'
    ch_vids = [line.replace('href="', 'youtube.com') for line in data if item in line]
    # remove dupes
    ch_vids = list(set(ch_vids))
    if GRAB_LAST > 0:
      # store urls, from most recent to GRAB_LAST
      vidURLs[name] = ch_vids[0:GRAB_LAST]
      if not os.path.exists(pathName):
          os.mkdir(pathName)
      # download videos
      for vid in vidURLs[name]:
          yt = YouTube(vid)
          vidInfo[name + yt.title] = yt

In [0]:
if not os.path.exists('videos'):
    os.mkdir('videos')

# open a newpipe subscription json
with open('subs.json') as f:
    json_data = json.load(f)
json_data = json_data['subscriptions']

# Put channel name:url in a dict
for item in json_data:
    for key, val in item.items():
        if key == 'name':
            name = val
        if key == 'url':
            url = val
            # append video list view sorted by recent
            url += "/videos?view=0&sort=dd&flow=list"
    subs[name] = url

In [14]:
for name,url in subs.items():
  # TODO learn to parallel process LOL
  # pool.map(getChannelVids, sub)
  getChannelVids(name, url, 1)

HTTPError: ignored